# Decoding Behavior from Behavior

### Import

In [ ]:
# FISSA: Signal Extraction & Source-Separation
# Load this module to organize loading our trace data
from AnalysisModules.FissaAnalysis import FissaModule

# CASCADE: Spike Inference
# Load this module to organize loading our inference data
from AnalysisModules.CascadeAnalysis import CascadeModule

# Behavioral Decoder
# Load this module to organize encoding-decoding analyses
# Interfaces Neural Decoding Package from Joshua Glaser via Kording Lab.
# Interfaces with State-Space Modeling (SSM / SSM-jax) Package from Scott Linderman Lab
# Interfaces with an annotated implementation of
# Manifold Inference from Neural Dynamics Package (MIND) from David Tank Lab
# Annotated/Implemented by Quentin RV Ferry via Tonegawa Lab
# Interfaces with PySindy Package from the Dynamics Lab at UW
from AnalysisModules.DecodingAnalysis import DecodingModule

# Visualization Packages
import matplotlib
#matplotlib inline
matplotlib.use('Qt5Agg')

### Load Data

In [ ]:
# Instantiate Traces & Associated Data
Data = FissaModule(data_folder="H:\\DEM_Excitatory_Study\\DEM2\\Retrieval\\Imaging\\10Hz", index_path="H:\\DEM_Excitatory_Study\\DEM2\\Retrieval\\Imaging\\10Hz\\NeuronalIndex.csv")
# Initialize
Data.initializeFissa()
# Load
Data.loadFissaPrep()
Data.loadFissaSep()
Data.loadProcessedTraces()

# Instantiate Firing Rates & Associated Data
Data.Cascade = CascadeModule(Data.ProcessedTraces.merged_dFoF_result, Data.frame_rate, model_folder="C:\\ProgramData\\Anaconda3\\envs\\suite2p\\Pretrained_models")
Data.Cascade.loadSpikeProb(load_path=Data.output_folder)
Data.Cascade.loadSpikeInference(load_path=Data.output_folder)
Data.Cascade.loadProcessedInferences(load_path=Data.output_folder)

In [ ]:
from AnalysisModules.DecodingAnalysis import WienerFilter
BD = WienerFilter(NeuralData=Data.ProcessedTraces.dFoF_result)